# Depletion 2: Analysis and Results

In this notebook, we will run a depletion simulation and then manually calculate the burnup to understand the physics.

We will cover:
1. **Running Depletion**: Setting up the operator and integrator.
2. **Manual Burnup Calculation**
3. **Isotopic Evolution**: Plotting how fuel is consumed and plutonium is bred.
4. **End of Cycle Composition**: Checking the final fuel state.

Please read the following before starting: https://docs.openmc.org/en/stable/usersguide/tallies.html#normalization-of-tally-results

In [ ]:
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
import openmc
import openmc.deplete
import openmc.data

## Model Configuration

We begin by loading a standard PWR pin cell model. 

> **Note**: In OpenMC depletion, we must explicitly define the **volume** of the depletable materials. The code uses this volume to convert tally results (reaction rates) into absolute numbers of atoms.

In [ ]:
# 1. Setup Model
model = openmc.examples.pwr_pin_cell()

# Define fuel volume (Radius ~0.39 cm)
fuel_radius = 0.39218
model.materials[0].volume = math.pi * fuel_radius**2

We need to track how much power is produced in the location which we are interested in. Because we only have one material in this pin example, we can just track the "kappa-fission" (see documentation) for the fuel material. If you have multiple materials, you will have to create multiple tallies for each. The kappa-fission tracks how much heat is deposited coming from fission events.

--> The recoverable energy production rate due to fission. The recoverable energy is defined as the fission product kinetic energy, prompt and delayed neutron kinetic energies, prompt and delayed γ-ray total energies, and the total energy released by the delayed β particles. The neutrino energy does not contribute to this response. The prompt and delayed γ-rays are assumed to deposit their energy locally. Units are eV per source particle.

In [ ]:
model.tallies = [openmc.Tally(name='heating')]
model.tallies[0].filters = [openmc.MaterialFilter(model.materials[0])]
model.tallies[0].scores = ['kappa-fission']

## 2. Depletion Parameters

To run the simulation, we need a **Depletion Chain** (which contains decay constants and fission yields) and an **Integrator**, which manages the time-stepping.

In [ ]:
# Simulation Settings
model.settings.particles = 1000
model.settings.inactive = 10
model.settings.batches = 50

# Depletion Settings
operator = openmc.deplete.CoupledOperator(model, "./chain_simple.xml")

# Schedule: 6 steps of 100 days each
time_steps = [100] * 6  # days
power = 100.0           # Watts (for this entire fuel pin slice)

integrator = openmc.deplete.PredictorIntegrator(operator, time_steps, power, timestep_units='d')
integrator.integrate()

## Analysis: Calculating Heavy Metal Mass

Burnup is conventionally expressed in Megawatt-days per kilogram of **Heavy Metal** ($MWd/kgHM$). Heavy metal refers to actinides with $Z \ge 90$ (primarily Uranium and Plutonium).

$$ M_{HM} = V_{fuel} \times \rho_{fuel} \times w_{HM} $$

Where:
* $V_{fuel}$ is the fuel volume.
* $\rho_{fuel}$ is the mass density.
* $w_{HM}$ is the weight fraction of heavy metal nuclides.

In [ ]:
# Get the fuel material (index 0 in pwr_pin_cell)
fuel = model.materials[0]

# 1. Total Mass of the Fuel (UO2)
# Density is in g/cm3, Volume in cm3
total_mass_g = fuel.volume * fuel.get_mass_density()

# 2. Calculate Heavy Metal Mass (Uranium only)
# We filter for nuclides with Atomic Number (Z) >= 90
hm_mass_g = 0.0
for nuclide in ["U235", "U238"]:
    hm_mass_g += fuel.get_mass(nuclide)

# Convert to kg
initial_hm_kg = hm_mass_g / 1000.0

print(f"Total Fuel Mass (UO2): {total_mass_g:.4f} g")
print(f"Heavy Metal Mass (U):  {hm_mass_g:.4f} g")
print(f"Initial HM Mass:       {initial_hm_kg:.6f} kg")

## Normalization and Heating Power: The $\kappa$-fission Approach

To convert the raw results from an OpenMC simulation into physical quantities like Watts or Joules, we must apply a normalization factor. As outlined in the [OpenMC Documentation (Section 8.3)](https://docs.openmc.org/en/stable/usersguide/tallies.html#normalization-of-tally-results), OpenMC tallies are typically normalized "per source neutron."

In depletion analysis, we use the **$\kappa$-fission** (kappa-fission) tally to determine the energy generated in the fuel. The recoverable energy production rate due to fission. The recoverable energy is defined as the fission product kinetic energy, prompt and delayed neutron kinetic energies, prompt and delayed γ-ray total energies, and the total energy released by the delayed β particles. The neutrino energy does not contribute to this response. The prompt and delayed γ-rays are assumed to deposit their energy locally. Units are eV per source particle.

## Understanding Burnup ($BU$)

**Burnup** is a measure of how much energy has been extracted from a primary fuel source. It is most commonly expressed in units of **Megawatt-days per kilogram of Heavy Metal** ($MWd/kgHM$).

### The Burnup Equation
The cumulative burnup at any time step $i$ is defined by the total energy produced divided by the initial mass of heavy metal:

$$BU_i = \frac{\sum_{j=0}^{i} (P_j \times \Delta t_j)}{M_{HM, 0}}$$

### Variable Definitions:
| Variable | Description | Units |
| :--- | :--- | :--- |
| **$P_j$** | The operating power during time step $j$. | $MW$ |
| **$\Delta t_j$** | The duration of the time step. | $days$ |
| **$M_{HM, 0}$** | Initial mass of **Heavy Metal** (actinides with $Z \ge 90$). | $kg$ |
| **$E$** | Cumulative thermal energy produced ($\sum P \Delta t$). | $MWd$ |

In [ ]:
sp_files = [f'openmc_simulation_n{i}.h5' for i in range(len(time_steps))]

joules_per_eV = 1.6021e-19
normalization_factor = []
burnup_list = [0.0]
cumulative_energ_joules = 0.0

# Constants
joules_per_eV = 1.60218e-19
seconds_per_day = 86400
mwd_to_joules = 8.64e10

burnup_list = [0.0]
cumulative_energy_joules = 0.0

for i, sp_name in enumerate(sp_files):
    with openmc.StatePoint(sp_name) as sp:
        tally = sp.get_tally(name='heating')
        # Score is in eV / source particle
        kappa_fiss_per_src = tally.get_values(scores=['kappa-fission']).sum()
        keff = sp.keff.n
        
        # Energy produced per source particle (Joules / source)
        energy_per_source = (kappa_fiss_per_src * joules_per_eV) / keff

        normalization_factor = power / energy_per_source
        if i > 0:
            delta_t_sec = time_steps[i] * seconds_per_day 
            
            # Energy (J) = Power (J/s) * Time (s)
            energy_interval = normalization_factor * energy_per_source * delta_t_sec
            cumulative_energy_joules += energy_interval

            # Burnup = Total Energy (MWd) / Heavy Metal Mass (kg)
            current_burnup = (cumulative_energy_joules / mwd_to_joules) / initial_hm_kg
            burnup_list.append(current_burnup)

In [ ]:
fig, ax = plt.subplots(figsize=(5,4), layout='constrained')

ax.plot(np.cumsum(time_steps), burnup_list, label='pin')

ax.set(
    xlabel='time, days',
    ylabel='burnup, MWd/kgHMi'
)

If there are multiple materials, we have to calculate the apply the following procedure:
1. Tally kappa-fission for all fissile materials in the model
2. Take sum of all kappa-fissions to get the total kappa-fission
3. Calculate the energy released per source from total kappa-fissions
4. Calculate normalization factor
5. Apply this normalization factor to the heat generated in each material (J/source) to get the energy released in a material.
6. Convert energy released in material to the burnup for each time step.

## Isotopic Evolution

We use the `depletion_results.h5` file to look at how the material composition changes.

In [ ]:
# Load Results
results = openmc.deplete.Results("./depletion_results.h5")

# Get Fuel Material ID
fuel = model.materials[0]

# Extract Atoms
# get_atoms(mat_id, nuclide) returns -> (time_array, atom_array)
times, u235 = results.get_atoms(fuel, "U235")
_, xe135 = results.get_atoms(fuel, "Xe135")

# Convert time to days
times_days = times / (24 * 3600)

# Plot
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Time [days]')
ax1.set_ylabel('U-235 Atoms', color=color)
ax1.plot(times_days, u235, color=color, linewidth=2, label="U-235")
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Xe-135 Atoms', color=color)
ax2.plot(times_days, xe135, color=color, linewidth=2, linestyle='--', label="Xe-135")
ax2.tick_params(axis='y', labelcolor=color)

plt.show()

## End of Cycle (EOC) Composition

Finally, we can print out the composition of the fuel at the last time step.

In [ ]:
results.export_to_materials(-1)[0]